In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#connecting to database
try:
    database_path = "data/data.db"
    conn = sqlite3.connect(database_path)
    query = open('data/fetch_all.sql', 'r').read()
    df = pd.read_sql_query(query, conn)
except (sqlite3.Error, FileNotFoundError) as e:
    print(f"An error occurred: {e}")
finally:
    if 'conn' in locals():
        conn.close()

# Load the second_phase_target.csv file
target_ids = pd.read_csv('second_phase_target.csv')

# Filter the main dataset based on target field in target_ids
predicted_df = df[df['client_id'].isin(target_ids['target'])].copy()
predicted_df.drop(columns=['account_id', 'campaign_id', 'campaign', 'outcome_id', 'previous', 'poutcome', 'pdays', 'month'], inplace=True)



# Binning age into common bins
age_bins = [0, 25, 36, 55, 65, 100]
age_labels = ['<25', '25-35', '36-55', '55-65', '65+']
predicted_df.loc[:, 'age_bin'] = pd.cut(predicted_df['age'], bins=age_bins, labels=age_labels, right=False)

predicted_df.head()

# # Summary statistics
# print("\nSummary statistics:")
# print(predicted_df.describe())
# print(predicted_df.info())

# # Distribution of numerical features
# numerical_features = predicted_df.select_dtypes(include=['int64', 'float64']).columns
# predicted_df[numerical_features].hist(bins=15, figsize=(15, 6))
# plt.suptitle('Distribution of Numerical Features')
# plt.show()


# # Distribution of categorical features
# categorical_features = predicted_df.select_dtypes(include=['object']).columns
# for feature in categorical_features:
#     plt.figure(figsize=(10, 4))
#     sns.countplot(data=predicted_df, x=feature)
#     plt.title(f'Distribution of {feature}')
#     plt.xticks(rotation=45)
#     plt.show()


,client_id,age,job,marital,education,in_default,balance,housing,loan,contact,day,duration,y,age_bin
259,4735367,40,blue-collar,married,secondary,no,475.0,yes,no,unknown,5,332,unknown,36-55
260,4258697,53,blue-collar,married,secondary,no,70.0,yes,no,unknown,5,611,no,36-55
319,7394433,42,blue-collar,married,secondary,no,490.0,yes,no,unknown,5,386,unknown,36-55
752,2040771,36,management,married,tertiary,no,144.0,yes,no,unknown,6,164,unknown,36-55
783,3927313,38,technician,single,secondary,no,42.0,yes,no,unknown,7,195,unknown,36-55


In [ ]:
# Add a column 'potential_deposit' calculated at 10% of the balance column
df_min_dep = predicted_df[predicted_df['balance'] * 0.10 >= 200].copy()
forecast_df = predicted_df.copy()

CNB_RATE = 0.03113 # NOTE: Czech National Bank rate paid on term deposits
CLIENT_RATE = 0.0275 #  NOTE: Market rate paid to clients on term deposits

predicted_df.head(20)
predicted_df['pred_deposit'] = predicted_df['balance'] * 0.10
predicted_df['CNB_rate'] = CNB_RATE
predicted_df['client_rate'] = CLIENT_RATE
predicted_df['gross_margin'] = predicted_df['pred_deposit'] * (predicted_df['CNB_rate'] - predicted_df['client_rate'])
predicted_df.head(20)

,client_id,age,job,marital,education,in_default,balance,housing,loan,contact,day,duration,y,age_bin,pred_deposit,CNB_rate,client_rate,gross_margin
259,4735367,40,blue-collar,married,secondary,no,475.0,yes,no,unknown,5,332,unknown,36-55,47.5,0.03113,0.0275,0.172425
260,4258697,53,blue-collar,married,secondary,no,70.0,yes,no,unknown,5,611,no,36-55,7.0,0.03113,0.0275,0.025410
319,7394433,42,blue-collar,married,secondary,no,490.0,yes,no,unknown,5,386,unknown,36-55,49.0,0.03113,0.0275,0.177870
752,2040771,36,management,married,tertiary,no,144.0,yes,no,unknown,6,164,unknown,36-55,14.4,0.03113,0.0275,0.052272
783,3927313,38,technician,single,secondary,no,42.0,yes,no,unknown,7,195,unknown,36-55,4.2,0.03113,0.0275,0.015246
793,2266010,31,blue-collar,single,secondary,no,53.0,yes,no,unknown,7,446,unknown,25-35,5.3,0.03113,0.0275,0.019239
818,2079564,37,technician,single,tertiary,no,147.0,no,no,unknown,7,416,unknown,36-55,14.7,0.03113,0.0275,0.053361
835,4078422,44,technician,divorced,secondary,no,-329.0,yes,no,unknown,7,171,no,36-55,-32.9,0.03113,0.0275,-0.119427
966,8529232,49,services,married,secondary,no,1496.0,yes,no,unknown,7,218,unknown,36-55,149.6,0.03113,0.0275,0.543048
997,9846155,43,blue-collar,married,primary,no,61.0,yes,no,unknown,7,261,unknown,36-55,6.1,0.03113,0.0275,0.022143
